# 1. Test Corpus
## 1.1 Initialize Corpus

In [ ]:
from pprint import pprint
from nlptext.base import BasicObject

########### ResumeNER ###########
CORPUSPath = 'dataset/ResumeNER/'
corpusFileIden = '.bmes'
textType   = 'block'
Text2SentMethod  = 're'
Sent2TokenMethod = 'iter'
TOKENLevel = 'char'
anno = 'embed'
annoKW = {}

MaxTextIdx = False

BasicObject.INIT(CORPUSPath, corpusFileIden, textType, 
                 Text2SentMethod, Sent2TokenMethod, TOKENLevel, 
                 anno, annoKW, MaxTextIdx)


from nlptext.corpus import Corpus
corpus = Corpus()
# corpus.IdxFolderStartEnd

print([i for i in corpus.TOKEN])


In [ ]:
ListTokenUnique, DictTokenUniqe = corpus.TokenUnique
print(ListTokenUnique)

## 1.2 Test in Corpus System


### 1.2.1 CTX_IND_CHANNELS
#### Token

In [ ]:
channel = 'stroke'

tk = corpus.Tokens[10]
print(tk.getChannelGrain(channel, end_grain=True))

#### Sentence

In [ ]:
channel = 'stroke'
st = corpus.Sentences[2]
print(st.getChannelGrain(channel, Max_Ngram=2, useStartEnd=True, end_grain=True))

### 1.2.2 CTX_DEP_CHANNELS

#### Token

In [ ]:
tk = corpus.Tokens[10]
print(tk)
print()
print(tk.getChannelGrain('stroke'))
print()
print(tk.getChannelGrain('subcomp', Max_Ngram=2, end_grain=True))
print()
print(tk.getChannelGrain('pos'))

#### Sentence

In [ ]:

st = corpus.Sentences[2]

print(st.getChannelGrain('stroke'))
print()
print(st.getChannelGrain('subcomp', end_grain=True))
print()
print(st.getChannelGrain('pos'))

print()
print(st.getChannelGrain('pos', Max_Ngram=2, useStartEnd=True))

### 1.2.3 ANNO_CHANNEL

#### Token

In [ ]:
# one applicable in the corpus text
channel = 'annoE'

tk = corpus.Tokens[10]
print(tk.getChannelGrain(channel, end_grain=True))

In [ ]:
channel = 'annoE'
st = corpus.Sentences[2]
print(st.sentence, '\n')
print(st.getChannelGrain(channel, useStartEnd=True), '\n')

channel = 'annoR'
st = corpus.Sentences[2]
print(st.getChannelGrain(channel, useStartEnd=True), '\n')

In [ ]:
channel = 'annoE'
list(zip(st.sentence, st.getChannelGrain(channel, useStartEnd=False)))

## 1.3 Test in New Sentence whose TOKENLevel is Word

In [ ]:
PAD   = '</pad>'
START = '</start>'
END   = '</end>'
UNK   = '</unk>'
specialTokens     = [ PAD, START, END, UNK ]
specialTokensDict = {PAD: 0, START: 1, END: 2, UNK : 3, }


from nlptext.utils.grain import getChannelGrain4Token, getChannelGrain4Sent

#### Token

In [ ]:
from nlptext.token import Token

channel = 'radical'
token = '病理性'


tk = Token(token = token)
print(tk ,'\n')
print(tk.getChannelGrain('radical', Max_Ngram=2),'\n')
print(tk.getChannelGrain('stroke'),'\n')
print(tk.getChannelGrain('stroke', end_grain=True),'\n')

#### Sentence

In [ ]:
from nlptext.sentence import Sentence

sent =  '心率 62 次 / 分 ， 律 齐 ， 各 瓣膜 区 未闻 及 病理性 杂音 。'.split(' ')
st = Sentence(sentence=sent, tokenLevel = 'word')
print(st,'\n')
print(st.sentence,'\n')
print(st.getChannelGrain('token'),'\n')
print(st.getChannelGrain('stroke'),'\n')
print(st.getChannelGrain('subcomp', useStartEnd=True, end_grain=True),'\n')
print(st.getChannelGrain('pos'))

#### Sentence's Token

In [ ]:
tk = st.Tokens[0]
print(tk)
print()
print(tk.getChannelGrain('stroke'))
print()
print(tk.getChannelGrain('stroke', end_grain=True))

In [ ]:
# can use POS here.
print(tk.getChannelGrain('pos')) 

# 2. Build `ListGrainUnique` for All Channels


## 2.1 Function `get_dictionary_ngram`

In [ ]:

PAD   = '</pad>'
START = '</start>'
END   = '</end>'
UNK   = '</unk>'

specialTokens     = [ PAD, START, END, UNK]

In [ ]:

LTU, DTU = corpus.TokenUnique



num_specialtokens = len(specialTokens)

LTU[:num_specialtokens] == specialTokens



In [ ]:

##################################################################################################LTU_LGU-LT

def get_LGU_or_LT(TokenUnique, channel= 'char',
                  Max_Ngram = 1, end_grain = False,
                  specialTokens = specialTokens):

    # ListGrainUnique = []
    

    LTU, DTU = TokenUnique
    num_specialtokens = len(specialTokens)
    assert LTU[:num_specialtokens] == specialTokens
    
    
    DictGrainUnique = {}
    new_grains = [stk for stk in specialTokens]
    # ListGrainUnique = ListGrainUnique + new_grains
    for stk in new_grains:
        DictGrainUnique[stk] = len(DictGrainUnique)

    LookUp = [[idx] for idx in range(num_specialtokens)]
    
    print('For channel: |', channel, '| build GrainUnique and LookUp')
    for idx, token in enumerate(LTU[num_specialtokens:]):
        ChN = getChannelGrain4Token(token, channel, Max_Ngram= Max_Ngram, end_grain = end_grain)
        # new_grains = list(set(ChN) - set(ListGrainUnique))
        new_grains = [i for i in set(ChN) if i not in DictGrainUnique]
        # print(new_grains)
        # ListGrainUnique = ListGrainUnique + new_grains
        for stk in new_grains:
            DictGrainUnique[stk] = len(DictGrainUnique)
        LookUp.append([DictGrainUnique.get(gr) for gr in ChN])
        if (idx + num_specialtokens) % 100000 == 0:
            print('\t\tFor Channel:', channel, '\t', idx + num_specialtokens, datetime.now())
            
    ListGrainUnique = list(DictGrainUnique.keys())
    assert ListGrainUnique[:num_specialtokens] == specialTokens
    assert len(LookUp) == len(LTU)
    return (ListGrainUnique, DictGrainUnique), LookUp
##################################################################################################LTU_LGU-LT


In [ ]:
# this function is used in getListGrainUnique

TU = corpus.TokenUnique
Max_Ngram = 2
channel = 'subcomp'
GU, LT = get_LGU_or_LT(TU,  Max_Ngram = Max_Ngram, channel=channel)
print(GU[0])

In [ ]:
tk_idx = 56

In [ ]:
TGU = GU[0]

[TGU[i] for i in LT[tk_idx]]

In [ ]:
token = LTU[tk_idx]

getChannelGrain4Token(token, channel, Max_Ngram= Max_Ngram)

## 2.2 `getListGrainUnique`


Much Codes in the function `getListGrainUnique` in the module `base.py` .

In [ ]:
from nlptext.corpus import Corpus

corpus = Corpus()


channel_setting_temp = {'token':{'use':         True,
                                 'Max_Ngram':   1,
                                 'end_grain':   False,
                                 }}
    

channel = 'token'
channel_setting = channel_setting_temp[channel]
use = channel_setting.pop('use')
GrainUnique = corpus.getGrainUnique(channel, **channel_setting)

In [ ]:

channel_setting_temp = {'subcomp':{'use':         True,
                                   'Max_Ngram':   3,
                                   'end_grain':   True,
                                   }}
    

channel = 'subcomp'
channel_setting = channel_setting_temp[channel]
use = channel_setting.pop('use')
GrainUnique = corpus.getGrainUnique(channel, **channel_setting)

In [ ]:

channel_setting_temp = {'pos':    {'use':         True,
                                   'Max_Ngram':  1,
                                   'tagScheme': 'BIO' }}
    

channel = 'pos'
channel_setting = channel_setting_temp[channel]
use = channel_setting.pop('use')
GrainUnique = corpus.getGrainUnique(channel, **channel_setting)

In [ ]:

channel_setting_temp = {'annoE':  {'use':         True,
                                   'Max_Ngram':  1,
                                   'tagScheme': 'BIO' }}
    

channel = 'annoE'
channel_setting = channel_setting_temp[channel]
use = channel_setting.pop('use')
GrainUnique = corpus.getGrainUnique(channel, **channel_setting)

In [ ]:

channel_setting_temp = {'annoE':  {'use':         True,
                                   'Max_Ngram':  1,
                                   'tagScheme': 'BIOES' }}
    

channel = 'annoE'
channel_setting = channel_setting_temp[channel]
use = channel_setting.pop('use')
GrainUnique = corpus.getGrainUnique(channel, **channel_setting)

# 3 Using the Predefined Grain List for CTX_DEP and ANNO Channels


For example, we have a list at the very begining

In [ ]:
ANNO_Tag_List = ['不确定', '修饰',  '检查', '疾病', '症状']


def getTagDict(TagList, tagScheme = 'BIO'):
    L = []
    suffices = ['-B', '-I']
    if 'O' in tagScheme:
        pref = specialTokens[:-1] + ['O']
    
    else:
        pref = specialTokens[:-1] # without UNK
    if 'E' in tagScheme:
        suffices = suffices + ['-E']
    if 'S' in tagScheme:
        suffices = suffices + ['-S']
    for tag in TagList:
        L.extend([tag+suff for suff in suffices])
    return pref + L
    
TagList = ANNO_Tag_List
tagScheme = 'BIO'
    
getTagDict(TagList, tagScheme = tagScheme)

# 4. Test for NLPText's `BUILD_GRAIN_DICT` with Given Channel Settings

In [ ]:


from pprint import pprint
from nlptext.base import BasicObject

########### ResumeNER ###########
CORPUSPath = 'dataset/ResumeNER/'
corpusFileIden = '.bmes'
textType   = 'block'
Text2SentMethod  = 're'
Sent2TokenMethod = 'iter'
TOKENLevel = 'char'
anno = 'embed'
annoKW = {}



BasicObject.INIT(CORPUSPath, corpusFileIden, textType, 
                 Text2SentMethod, Sent2TokenMethod, TOKENLevel, 
                 anno, annoKW)

from nlptext.corpus import Corpus
corpus = Corpus()
# corpus.IdxFolderStartEnd


In [ ]:

import os
import shutil


from nlptext.corpus import Corpus


corpus = Corpus()
print(corpus.TokenNum_Dir)


CHANNEL_SETTINGS_TEMPLATE = {
    
    # CTX_IND
    'token':  {'use':         True,
               'Max_Ngram':   1},
    
    'char':   {'use':         True,
               'Max_Ngram':   1,
               'end_grain':   False},
    
    'basic':  {'use':         True,
               'Max_Ngram':   1,
               'end_grain':   False},
        
    'medical':{'use':         True,
               'Max_Ngram':   1,
               'end_grain':   False},
    
    'radical':{'use':         True,
               'Max_Ngram':   1,
               'end_grain':   False},
    
    'subcomp':{'use':         True,
               'Max_Ngram':   3,
               'end_grain':   True},

    'stroke': {'use':         True,
               'Max_Ngram':   5,
               'end_grain':   False},
    
    # CTX_DEP
    'pos':    {'use':         True,
               'Max_Ngram':   1,
               'end_grain':   False,
               'tagScheme':   'BIO',
              },
    
    # ANNO
    'annoR':  {'use':         True,
               'Max_Ngram':   1,
               'end_grain':   False,
               'tagScheme':   'BIO',
               },
    
    
    'annoE': {'use':         True,
              'Max_Ngram':   1,
              'end_grain':   False,
              'tagScheme':   'BIO',
              },
}


BasicObject.BUILD_GRAIN_UNI_AND_LOOKUP(CHANNEL_SETTINGS_TEMPLATE=CHANNEL_SETTINGS_TEMPLATE)


# TODO: pretty print the result.


### List Grain Unique

In [ ]:
print(corpus.GRAIN_UNI[corpus.TokenNum_Dir]['subcomp3e'][0])

In [ ]:
print(corpus.GRAIN_UNI[corpus.TokenNum_Dir]['medical'][0])

In [ ]:
print(corpus.GRAIN_UNI[corpus.TokenNum_Dir]['stroke5'][0])

In [ ]:
print(corpus.GRAIN_UNI[corpus.TokenNum_Dir]['pos'][0])

In [ ]:
print(corpus.GRAIN_UNI[corpus.TokenNum_Dir]['annoE'][0])

In [ ]:
print(corpus.GRAIN_UNI[corpus.TokenNum_Dir]['token'][0])

### LOOKUP

In [ ]:
[i for i in corpus.LOOKUP[corpus.TokenNum_Dir]]#['subcomp3']

In [ ]:
corpus.LOOKUP[corpus.TokenNum_Dir]['stroke5']

In [ ]:
from nlptext.token import Token

tk = Token(20)

tk

In [ ]:
channel_dict = {'channel': 'subcomp', 
               'Max_Ngram':   2,
               'end_grain':   False}

tk.getGrainTensor(dontUseLookUp = True, **channel_dict)

In [ ]:
tk.getGrainTensor(dontUseLookUp = True, **channel_dict)

In [ ]:
tk.getChannelGrain(**channel_dict)

# Wiki

In [ ]:


from pprint import pprint
from nlptext.base import BasicObject


########### Wiki ###########
CORPUSPath = 'dataset/WikiTotal/'
corpusFileIden = '.txt'

textType   = 'line'

Text2SentMethod  = 're'
Sent2TokenMethod = 'sep- '
TOKENLevel = 'char'

anno = False
annoKW = {}



MaxTextIdx = False

BasicObject.INIT(CORPUSPath, corpusFileIden, textType, 
                 Text2SentMethod, Sent2TokenMethod, TOKENLevel, 
                 anno, annoKW, MaxTextIdx)



In [ ]:




BOB = 'channel/WikiTotal/word/Token4692189/BOBMeta.p'
LGU = 'channel/WikiTotal/word/Token4692189/LGUDict.p'
BasicObject.INIT_FROM_PICKLE(BOB, LGU)



In [ ]:
from nlptext.token import Token
from nlptext.sentence import Sentence

st = Sentence(11)
st

In [ ]:
st.sentence

In [ ]:
[i for i in BasicObject.TOKEN]

In [ ]:
len(BasicObject.TEXT['NUMSents'])

In [ ]:
len(BasicObject.SENT['NUMTokens'])

In [ ]:
len(BasicObject.TOKEN['ORIGTokenIndex'])

In [ ]:


from pprint import pprint
from nlptext.base import BasicObject
from nlptext.utils import reCutText2Sent


########### Wiki ###########
CORPUSPath = 'dataset/wiki/'
corpusFileIden = '.txt'

textType   = 'line'

Text2SentMethod  = reCutText2Sent
Sent2TokenMethod = 'sep- '
TOKENLevel = 'char'

anno = False
annoKW = {}



MaxTextIdx = False

BasicObject.INIT(CORPUSPath, corpusFileIden, textType, 
                 Text2SentMethod, Sent2TokenMethod, TOKENLevel, 
                 anno, annoKW, MaxTextIdx)



In [ ]:
from nlptext.token import Token
from nlptext.sentence import Sentence

st = Sentence(17)
st

In [ ]:
st.sentence

In [ ]:
st.Tokens